In [174]:
import pandas as pd

In [175]:
data = pd.read_csv(r"/home/jamesholcombe/git/title-one/api/api/data/video_games.csv")

In [176]:
data.columns

Index(['Title', 'Features.Handheld?', 'Features.Max Players',
       'Features.Multiplatform?', 'Features.Online?', 'Metadata.Genres',
       'Metadata.Licensed?', 'Metadata.Publishers', 'Metadata.Sequel?',
       'Metrics.Review Score', 'Metrics.Sales', 'Metrics.Used Price',
       'Release.Console', 'Release.Rating', 'Release.Re-release?',
       'Release.Year', 'Length.All PlayStyles.Average',
       'Length.All PlayStyles.Leisure', 'Length.All PlayStyles.Median',
       'Length.All PlayStyles.Polled', 'Length.All PlayStyles.Rushed',
       'Length.Completionists.Average', 'Length.Completionists.Leisure',
       'Length.Completionists.Median', 'Length.Completionists.Polled',
       'Length.Completionists.Rushed', 'Length.Main + Extras.Average',
       'Length.Main + Extras.Leisure', 'Length.Main + Extras.Median',
       'Length.Main + Extras.Polled', 'Length.Main + Extras.Rushed',
       'Length.Main Story.Average', 'Length.Main Story.Leisure',
       'Length.Main Story.Median', 'Le

In [177]:
import requests

In [178]:
CLIENT_ID = "anpiyt29ez53n9ust8zice1jte3nde"
CLIENT_SECRET = "xrp3toczg1gi11pgun9m3plsujr8mw"

r = requests.post("https://id.twitch.tv/oauth2/token", params={
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
})


TOKEN = r.json()["access_token"]
AUTH_HEADER = {"Authorization": f"Bearer {TOKEN}", "Client-ID": CLIENT_ID}

In [179]:
query = """
fields cover.url,
first_release_date,
genres.name,
involved_companies.company.name, 
summary,
name,
age_ratings.rating,
genres.name,
involved_companies.company.logo.url;
limit 500; 
where (age_ratings.rating = (12, 15, 16, 17, 18)) & summary != null;
"""


r = requests.post("https://api.igdb.com/v4/games", headers= AUTH_HEADER ,data = query)
r.raise_for_status()

games = r.json()
# df_games = pd.json_normalize(games, max_level=3, meta = [["genres", "name"], "involved_companies"])
# df_games.head()

# df_games["involved_companies"] = df_games["involved_companies"].apply(lambda x: x[0]["company"] if hasattr(x, "__getitem__") else None)
# df_games


def take_first(data : "list[dict]", key : str):
    for i,d in enumerate(data):
        if not isinstance(d, dict):
            del data[i]

    
        entry= d.get(key, None)
        if entry is  None:
            del data[i]
        try:
            d[key] = entry[0]
        except (TypeError, IndexError):
            del data[i]
    
    return data

games = take_first(games, "genres")
games = take_first(games, "involved_companies")
games = take_first(games, "age_ratings")


df_games = pd.json_normalize(games)






In [180]:
df_games.head()
df_games.columns

Index(['id', 'first_release_date', 'name', 'summary', 'age_ratings.id',
       'age_ratings.rating', 'cover.id', 'cover.url', 'genres.id',
       'genres.name', 'involved_companies.id', 'involved_companies.company.id',
       'involved_companies.company.logo.id',
       'involved_companies.company.logo.url',
       'involved_companies.company.name', 'genres', 'involved_companies'],
      dtype='object')

In [181]:
df_games.rename(columns={
    'involved_companies.company.logo.url': "developer_logo_url",
    'involved_companies.company.name': "developer",
    'genres.name': "category",
    'age_ratings.rating': "age_rating",
    'first_release_date': "release_date",
    'cover.url': "image_url",
    'summary': "description",

}, inplace=True)
df_games["short_description"] = df_games["description"].apply(lambda x: x.split(".")[0])
df_games["long_description"] = df_games["description"]


In [183]:
df_games = df_games[["name", "short_description", "long_description", "description", "age_rating", "image_url", "category", "release_date", "developer", "developer_logo_url"]]


In [186]:
df_games.to_csv(r"games.csv", index=False)